In [1]:
import numpy as np
import pandas as pd

# Settings

In [30]:
settings = { 
            'data_size': int(1e2), # size of data subset after shuffle is performed
            'test_size': 0.25, # fraction of data set to be assigned as test data
            'save_env': False, # save environment
            'del': False, # Delete variables that are no longer needed to proceed in computations to save place
            'learning_curve': { # parameters to generate learning_curve
                'start': int(5e4), # training set start size (including)
                'stop': int(1e6 + 2.5e4), # traing set stop size (excluding)
                'step': int(2.5e4)  # increase between iterations
                              }
           }

# Load Data: load dataset, shuffle it and take subset

In [3]:
PATH = "../data/"

In [4]:
data_full = pd.read_csv(f'{PATH}train.tsv', sep='\t')

In [5]:
data_full.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [6]:
len(data_full)

1482535

In [7]:
from sklearn.utils import shuffle
data_shuffled = shuffle(data_full)

In [8]:
if (settings['del']):
    del data_full

In [9]:
data = data_shuffled.iloc[:settings['data_size'], :]

In [10]:
if (settings['del']):
    del data_shuffled

In [11]:
len(data)

100

In [12]:
data.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
683158,683158,Bullhead Jeans,2,"Women/Jeans/Slim, Skinny",Bullhead,14.0,0,"Bullhead Brand from Pacsun. Size 3. High Rise,..."
122724,122724,Joop (M) perfume body oil,1,Beauty/Fragrance/Men,NaN,6.0,1,"1/3 oz Joop (M) perfume body oil, roll on bott..."
680238,680238,BLACK & WHITE MAXI DRESS,2,Women/Dresses/Full-Length,FOREVER 21,16.0,0,BLACK & WHITE MAXI DRESS
629486,629486,Apple stuff,2,Home/Home Décor/Home Décor Accents,NaN,0.0,0,"Red apple cookie jar, coffee pot with matching..."
799513,799513,Hot pink slime,2,Kids/Toys/Arts & Crafts,Elmers,7.0,0,No description yet


# Data preparation

## Item Category

Split category_name into main_cat, subcat_1 and subcat_2

In [13]:
# reference: BuryBuryZymon at https://www.kaggle.com/maheshdadhich/i-will-sell-everything-for-free-0-55
def split_cat(text):
    try: return text.split("/")
    except: return ("No Label", "No Label", "No Label")

In [14]:
data.loc[:,'main_cat'], data.loc[:,'subcat_1'], data.loc[:,'subcat_2'] = \
zip(*data.loc[:,'category_name'].apply(lambda x: split_cat(x)))
# data.head()

In [15]:
data.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,main_cat,subcat_1,subcat_2
683158,683158,Bullhead Jeans,2,"Women/Jeans/Slim, Skinny",Bullhead,14.0,0,"Bullhead Brand from Pacsun. Size 3. High Rise,...",Women,Jeans,"Slim, Skinny"
122724,122724,Joop (M) perfume body oil,1,Beauty/Fragrance/Men,NaN,6.0,1,"1/3 oz Joop (M) perfume body oil, roll on bott...",Beauty,Fragrance,Men
680238,680238,BLACK & WHITE MAXI DRESS,2,Women/Dresses/Full-Length,FOREVER 21,16.0,0,BLACK & WHITE MAXI DRESS,Women,Dresses,Full-Length
629486,629486,Apple stuff,2,Home/Home Décor/Home Décor Accents,NaN,0.0,0,"Red apple cookie jar, coffee pot with matching...",Home,Home Décor,Home Décor Accents
799513,799513,Hot pink slime,2,Kids/Toys/Arts & Crafts,Elmers,7.0,0,No description yet,Kids,Toys,Arts & Crafts


## Overview

In [16]:
print("Number of unique fields:\n")

print("main_cat: \t%d" % data['main_cat'].nunique())
print("subcat_1: \t%d" % data['subcat_1'].nunique())
print("subcat_2: \t%d" % data['subcat_2'].nunique())
print("brand_name: \t%d" % data['brand_name'].nunique())
print()

print("%d items have no category" % len(data.loc[data['main_cat'] == 'No Label']))
print("%d items have no brand" % data['brand_name'].isna().sum())

Number of unique fields:

main_cat: 	11
subcat_1: 	37
subcat_2: 	64
brand_name: 	48

1 items have no category
39 items have no brand


## Numerically represent features

### train_id: copy

Create new DataFrame called data_num for numerical representations

In [17]:
data_num = pd.DataFrame(data.loc[:,'train_id'], columns=['train_id'])

### name: represent name by its length as name_len

In [18]:
data_num.loc[:,'name_len'] = data['name'].str.len()

### item_condition_id, price, shipping: copy

In [19]:
data_num[['item_condition_id', 'price', 'shipping']] = data.loc[:,['item_condition_id', 'price', 'shipping']]

### item_description

Represent item_description by its length as item_description_len

In [20]:
item_description_len = data.loc[:,'item_description'].str.len()
data_num['item_description_len'] = item_description_len

# Replace NaN in data_num.item_description_len column by zeros
data_num['item_description_len'] = data_num['item_description_len'].fillna(0)

# Change data type of this column to uint16 provided the max val is less than 65535
if (data_num.item_description_len.max() < 65535):
    data_num['item_description_len'] = data_num['item_description_len'].astype(np.uint16)

In [21]:
# data_num.head()

In [22]:
# data.head()

### Define function for making binary columns

In [23]:
def make_binary_columns(df_str, df_num, column_name):
    """
    Turns a single column named column_name (with various categories) into m binary columns, where m is the number
    of unique values in the original column. For each sample, the value for all new columns is 0 apart the one
    that matches the value of the original column. Names of new binary columns are formed as follows:
    column_name + '_' + str(original column value)
    Inputs:
        two pandas DataFrames: df_str where a single column contains information about given category
                               df_num that will later contain many binary columns
        column_name: name of the column that will be split into several binary columns
    Returns nothing. It appends the collumns directly into df_num to increase performance.
    """
    m = len(df_str)
    if (m != len(df_num)):
        raise Exception("df_str and df_num must have the same size.")
 
    categories = df_str[column_name].unique()

    from tqdm import tqdm_notebook # progress bar

    # Create a new feature for each category and initialize it to 0
    for i in tqdm_notebook(categories, desc='1/2'):
        df_num[column_name + '_' + str(i)] = np.zeros((m, 1), dtype=np.int8)

#     Loop thorugh all rows and assign 1 to the column whose name is the same as category
    for i in tqdm_notebook(df_str.index, desc='2/2'): # loop through all rows
        category = str(df_str.at[i, column_name])
        df_num.at[i, column_name + '_' + category] = 1

### brand_name: for each unique one create new binary feature

In [24]:
make_binary_columns(data, data_num, 'brand_name')

In [25]:
# data.head()

### main_cat, subcat_1, subcat_2: for each unique one create new binary feature

In [26]:
make_binary_columns(data, data_num, 'main_cat')
make_binary_columns(data, data_num, 'subcat_1')
make_binary_columns(data, data_num, 'subcat_2')

In [27]:
if (settings['del']):
    del data

# Split data and extract X, y and train_id

In [28]:
from sklearn.model_selection import train_test_split
X_train_unscaled, X_test_unscaled = train_test_split(data_num, test_size = settings['test_size'], random_state=None) # randomly split data
# ! X_train_unscaled and X_test_unscaled STILL CONTAINS PRICE AT THIS MOMENT !

In [29]:
if (settings['del']):
    del data_num

In [31]:
# Pop price from X_train and X_test
y_train = X_train_unscaled.pop('price')
y_test = X_test_unscaled.pop('price')

In [32]:
# Pop id_train from both training and test data set

id_train = X_train_unscaled.pop('train_id')
id_test = X_test_unscaled.pop('train_id')

# Scale data

In [33]:
from sklearn.preprocessing import MaxAbsScaler

X_train = X_train_unscaled.copy()
X_test = X_test_unscaled.copy()

columns_to_scale = ['name_len', 'item_condition_id', 'item_description_len']

scaler = MaxAbsScaler()
scaler.fit(X_train_unscaled[columns_to_scale]) # Compute the mean adn std of training data to be used for future scaling

X_train[columns_to_scale] = pd.DataFrame(scaler.transform(X_train_unscaled[columns_to_scale]), index=X_train_unscaled.index, columns=columns_to_scale)
if (settings['del']):
    del X_train_unscaled

X_test[columns_to_scale] = pd.DataFrame(scaler.transform(X_test_unscaled[columns_to_scale]), index=X_test_unscaled.index, columns=columns_to_scale)
if (settings['del']):
    del X_test_unscaled

In [34]:
X_train_unscaled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 34025 to 199661
Columns: 165 entries, name_len to subcat_2_Personal Care
dtypes: int64(3), int8(161), uint16(1)
memory usage: 14.3 KB


In [35]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 34025 to 199661
Columns: 165 entries, name_len to subcat_2_Personal Care
dtypes: float64(3), int64(1), int8(161)
memory usage: 14.7 KB


In [36]:
X_test_unscaled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 1077763 to 1285799
Columns: 165 entries, name_len to subcat_2_Personal Care
dtypes: int64(3), int8(161), uint16(1)
memory usage: 4.8 KB


In [37]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 1077763 to 1285799
Columns: 165 entries, name_len to subcat_2_Personal Care
dtypes: float64(3), int64(1), int8(161)
memory usage: 4.9 KB


In [40]:
X_test.head()

,name_len,item_condition_id,shipping,item_description_len,brand_name_Bullhead,brand_name_nan,brand_name_FOREVER 21,brand_name_Elmers,brand_name_Nike,brand_name_PUMA,...,subcat_2_Eyes,subcat_2_Flats,subcat_2_One-Pieces,subcat_2_Blouse,subcat_2_Lanyard,subcat_2_Christian Books & Bibles,subcat_2_Two-Piece,subcat_2_Totes & Shoppers,subcat_2_Embroidery,subcat_2_Personal Care
1077763,0.750,0.75,0,0.091371,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1249004,0.400,0.50,1,0.042640,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
836377,0.325,0.75,0,0.039594,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1282944,0.425,0.75,1,0.190863,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
629486,0.275,0.50,0,0.101523,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
X_test_unscaled.head()

,name_len,item_condition_id,shipping,item_description_len,brand_name_Bullhead,brand_name_nan,brand_name_FOREVER 21,brand_name_Elmers,brand_name_Nike,brand_name_PUMA,...,subcat_2_Eyes,subcat_2_Flats,subcat_2_One-Pieces,subcat_2_Blouse,subcat_2_Lanyard,subcat_2_Christian Books & Bibles,subcat_2_Two-Piece,subcat_2_Totes & Shoppers,subcat_2_Embroidery,subcat_2_Personal Care
1077763,30,3,0,90,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1249004,16,2,1,42,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
836377,13,3,0,39,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1282944,17,3,1,188,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
629486,11,2,0,100,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# X_train.to_csv('X_train.csv')
# X_train_scaled.to_csv('X_train_scaled.csv')
# X_test.to_csv('X_test.csv')
# X_test_scaled.to_csv('X_test_scaled.csv')

In [ ]:
if (settings['save_env']):
    import dill                            #pip install dill --user
    dill.dump_session('splittedData.pkl')

# Check size of variables

In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' By Fred Cirera, after https://stackoverflow.com/a/1094933/1870254'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name,value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name,sizeof_fmt(size)))

# Performance measure: RMSE

$$\text{RMSE} \left( \mathbf{Y} , \mathbf{\hat{Y}} \right) = \sqrt{\frac{1}{n} \sum_{i=1}^n \left( y_i - \hat{y_i} \right)^2 } $$

In [ ]:
def rmse(y_test, y_pred):
    from sklearn.metrics import mean_squared_error
    return np.sqrt(mean_squared_error(y_test, y_pred))

# Learning curve for linear regression

In [ ]:
X_train_scaled[:2]

In [ ]:
from tqdm import tqdm_notebook # progress bar

for m in tqdm_notebook(range(settings['learning_curve']['start'], settings['learning_curve']['stop'], settings['learning_curve']['step'])):
    # slice dataset
    X_train_red = X_train[:m]
    y_train_red = y_train[:m]
    from sklearn.linear_model import LinearRegression
    
    
    lnr_regr = LinearRegression(n_jobs=-1)
    print("Trainig for training set size of: ", m)
    lnr_regr.fit(X_train_scaled[:m], y_train[:m])
    pred_train = lnr_regr.predict(X_train_scaled[:m])

In [ ]:
X_train_scaled.info()

In [ ]:
import datetime
filename = datetime.datetime.now().strftime('%y%m%d') + '_' + 'learning_curve.csv' # create file name starting with yymmdd_
with open('generated_data/' + filename, 'w') as f:
    f.write('training_set_size,trainin_error,test_error\n')

In [ ]:
with open('generated_data/' + filename, 'a') as f:
    f.write('1,1,2\n')

# Apply linear regression

In [ ]:
# Fit linear regression
from sklearn.linear_model import LinearRegression
lnr_regr = LinearRegression(n_jobs=-1)
print("Trainig...")
lnr_regr.fit(X_train_scaled, y_train)
print("Training done.")

if (settings['save_env']):
    dill.dump_session('linearModel.pkl')
    
# Make predictions and report train and test RMSEs

print("Evaluating performance on the training set...")
pred_train = lnr_regr.predict(X_train_scaled)
rmse_train = rmse(y_train, pred_train)
print("Training set RMSE: %.2f" % rmse_train)

print("Evaluating performance on the test test...")
pred_test = lnr_regr.predict(X_test_scaled)
rmse_test = rmse(y_test, pred_test)
print("Test set RMSE: %.2f" % rmse_test)

In [ ]:
# import dill

In [ ]:
# dill.load_session('linearModel.pkl')

In [ ]:
# who